In [92]:
import requests
import json
from pprint import pprint
import numpy as np
import pandas as pd
import os
import csv
import datetime as dt
import openweathermapy as ow # API call to get city long lat from city name

import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

from opencage.geocoder import OpenCageGeocode # used to get long and lat of given city name
from math import radians, sin, cos, asin, sqrt # used to calculate distance of room and city longs and lats

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

# Google API Key
from config import gkey


from flask import Flask, jsonify

In [2]:
results = requests.get('https://public.opendatasoft.com/api/records/1.0/download/?dataset=air-bnb-listings&rows=100&format=json').json()


In [3]:
pprint(results)

[{'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 292,
             'calculated_host_listings_count': 7,
             'city': 'Lisbon',
             'column_10': 125,
             'column_19': 'Portugal',
             'column_20': 'Portugal, Lisbon, Avenidas Novas',
             'coordinates': [38.74584282796673, -9.152359763007405],
             'host_id': 15282851,
             'id': 2998105,
             'last_review': '2019-04-17',
             'minimum_nights': 2,
             'name': 'Exciting Lisbon Entrecampos',
             'neighbourhood': 'Avenidas Novas',
             'number_of_reviews': 297,
             'reviews_per_month': 4.97,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-04-22'},
  'geometry': {'coordinates': [-9.152359763007405, 38.74584282796673],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': '64d5c2753e54fa16731ea8f7cf03163b079eb035'},
 {'datasetid': 'air-bnb-listing

 {'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 283,
             'calculated_host_listings_count': 3,
             'city': 'Lisbon',
             'column_10': 130,
             'column_19': 'Portugal',
             'column_20': 'Portugal, Lisbon, Santo Antnio',
             'coordinates': [38.72675852686154, -9.147616482164239],
             'host_id': 24755412,
             'id': 4805505,
             'last_review': '2018-11-20',
             'minimum_nights': 3,
             'name': 'Casa Pimenta - Home away from home',
             'neighbourhood': 'Santo Antnio',
             'number_of_reviews': 28,
             'reviews_per_month': 0.57,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-04-22'},
  'geometry': {'coordinates': [-9.147616482164239, 38.72675852686154],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': '89573a4c975aa297303d265ff149e97c4596b3dd'},
 {'datasetid': 'air-bnb-listi

             'host_id': 15803788,
             'id': 6298021,
             'last_review': '2019-04-07',
             'minimum_nights': 1,
             'name': 'Sunny Room in Bairro Alto/Bica',
             'neighbourhood': 'Misericrdia',
             'number_of_reviews': 201,
             'reviews_per_month': 4.22,
             'room_type': 'Private room',
             'updated_date': '2019-04-22'},
  'geometry': {'coordinates': [-9.15064562436264, 38.71090224406195],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': '4b25a602c9813029d83dfdced0f87ccd5aa90203'},
 {'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 70,
             'calculated_host_listings_count': 7,
             'city': 'Lisbon',
             'column_10': 37,
             'column_19': 'Portugal',
             'column_20': 'Portugal, Lisbon, Misericrdia',
             'coordinates': [38.71504998696641, -9.152508802232378],
             'host_id': 32184037,
      

  'fields': {'availability_365': 258,
             'calculated_host_listings_count': 3,
             'city': 'Lisbon',
             'column_10': 65,
             'column_19': 'Portugal',
             'column_20': 'Portugal, Lisbon, Santa Maria Maior',
             'coordinates': [38.71348784995498, -9.13642797560232],
             'host_id': 21173559,
             'id': 7678193,
             'last_review': '2019-02-16',
             'minimum_nights': 3,
             'name': 'Chão Loureiro apartment Baixa-Castle',
             'neighbourhood': 'Santa Maria Maior',
             'number_of_reviews': 93,
             'reviews_per_month': 2.1,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-04-22'},
  'geometry': {'coordinates': [-9.13642797560232, 38.71348784995498],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': '585fddfcf143e5b38fdc77f99cc3490c7da432f8'},
 {'datasetid': 'air-bnb-listings',
  'fields': {'availab

In [4]:
results = [r["fields"] for r in results]
results

[{'updated_date': '2019-04-22',
  'reviews_per_month': 4.97,
  'calculated_host_listings_count': 7,
  'city': 'Lisbon',
  'minimum_nights': 2,
  'coordinates': [38.74584282796673, -9.152359763007405],
  'room_type': 'Entire home/apt',
  'number_of_reviews': 297,
  'column_10': 125,
  'neighbourhood': 'Avenidas Novas',
  'last_review': '2019-04-17',
  'host_id': 15282851,
  'column_19': 'Portugal',
  'availability_365': 292,
  'id': 2998105,
  'column_20': 'Portugal, Lisbon, Avenidas Novas',
  'name': 'Exciting Lisbon Entrecampos'},
 {'updated_date': '2019-04-22',
  'city': 'Lisbon',
  'calculated_host_listings_count': 1,
  'minimum_nights': 1,
  'coordinates': [38.70213919994125, -9.177363170810427],
  'room_type': 'Private room',
  'number_of_reviews': 0,
  'column_10': 500,
  'neighbourhood': 'Alcntara',
  'host_id': 15307946,
  'column_19': 'Portugal',
  'availability_365': 0,
  'id': 3003712,
  'column_20': 'Portugal, Lisbon, Alcntara',
  'name': 'Fantastic Apartment with deck 120m

In [133]:
results_df = pd.DataFrame(results)
results_df.head()

,availability_365,calculated_host_listings_count,city,column_10,column_19,column_20,coordinates,host_id,id,last_review,minimum_nights,name,neighbourhood,number_of_reviews,reviews_per_month,room_type,updated_date
0,292,7,Lisbon,125,Portugal,"Portugal, Lisbon, Avenidas Novas","[38.74584282796673, -9.152359763007405]",15282851,2998105,2019-04-17,2,Exciting Lisbon Entrecampos,Avenidas Novas,297,4.97,Entire home/apt,2019-04-22
1,0,1,Lisbon,500,Portugal,"Portugal, Lisbon, Alcntara","[38.70213919994125, -9.177363170810427]",15307946,3003712,NaN,1,Fantastic Apartment with deck 120m2,Alcntara,0,NaN,Private room,2019-04-22
2,0,1,Lisbon,650,Portugal,"Portugal, Lisbon, So Domingos de Benfica","[38.74499852902329, -9.168202490560926]",15330291,3008031,NaN,1,Champions League apartment,So Domingos de Benfica,0,NaN,Entire home/apt,2019-04-22
3,0,1,Lisbon,500,Portugal,"Portugal, Lisbon, Algs, Linda-a-Velha e Cruz Q...","[38.70144477298908, -9.240587547103432]",15415294,3025693,NaN,1,Quarto em Algés,"Algs, Linda-a-Velha e Cruz Quebrada-Dafundo",0,NaN,Private room,2019-04-22
4,277,3,Lisbon,37,Portugal,"Portugal, Lisbon, Misericrdia","[38.71109888002856, -9.148347820132365]",15580486,3063405,2019-04-19,2,Camões Lounge Apartment w/Courtyard,Misericrdia,299,5.02,Entire home/apt,2019-04-22


In [134]:
cleanData = results_df[[ 'column_10','id', 'coordinates', 'room_type', "city", 'column_19']]
cleanData.head()

,column_10,id,coordinates,room_type,city,column_19
0,125,2998105,"[38.74584282796673, -9.152359763007405]",Entire home/apt,Lisbon,Portugal
1,500,3003712,"[38.70213919994125, -9.177363170810427]",Private room,Lisbon,Portugal
2,650,3008031,"[38.74499852902329, -9.168202490560926]",Entire home/apt,Lisbon,Portugal
3,500,3025693,"[38.70144477298908, -9.240587547103432]",Private room,Lisbon,Portugal
4,37,3063405,"[38.71109888002856, -9.148347820132365]",Entire home/apt,Lisbon,Portugal


In [137]:
cleanData = cleanData.rename(index=str, columns={"column_10":"Price",
                                                 "id":"Room_Id", 
                                                 "coordinates":"Coordinates",
                                                 "column_19":"Country", 
                                                 "room_type":"Room_Type",
                                                 "city":"City"})

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Room_Lat,Room_Long
0,125,2998105,"[38.74584282796673, -9.152359763007405]",Entire home/apt,Lisbon,Portugal,38.745843,-9.152360
1,500,3003712,"[38.70213919994125, -9.177363170810427]",Private room,Lisbon,Portugal,38.702139,-9.177363
2,650,3008031,"[38.74499852902329, -9.168202490560926]",Entire home/apt,Lisbon,Portugal,38.744999,-9.168202
3,500,3025693,"[38.70144477298908, -9.240587547103432]",Private room,Lisbon,Portugal,38.701445,-9.240588
4,37,3063405,"[38.71109888002856, -9.148347820132365]",Entire home/apt,Lisbon,Portugal,38.711099,-9.148348


In [140]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cleanData['Coordinates']:
   # Try to,
   try:
       # Split the row by comma and append
       # everything before the comma to lat
       lat.append(row[0])
       # Split the row by comma and append
       # everything after the comma to lon
       lon.append(row[1])
   # But if you get an error
   except:
       # append a missing value to lat
       lat.append(np.NaN)
       # append a missing value to lon
       lon.append(np.NaN)

# Create two new columns from lat and lon
cleanData['Room_Lat'] = lat
cleanData['Room_Long'] = lon


In [154]:
# cleanData = cleanData.drop(columns=["Coordinates"])
# cleanData.head()

In [142]:
cleanData["Selection"] = cleanData["City"]+", "+cleanData["Country"]
# cleanData.head(5)

In [148]:
cleanData['City'] = cleanData['City'].str.replace('-',' ')

,Price,Room_Id,Coordinates,Room_Type,City,Country,Room_Lat,Room_Long,Selection,City_Lat,City_Long,Calc_Distance
0,125,2998105,"[38.74584282796673, -9.152359763007405]",Entire home/apt,Lisbon,Portugal,38.745843,-9.152360,"Lisbon, Portugal",38.722252,-9.139337,1.774650
1,500,3003712,"[38.70213919994125, -9.177363170810427]",Private room,Lisbon,Portugal,38.702139,-9.177363,"Lisbon, Portugal",38.722252,-9.139337,2.476753
2,650,3008031,"[38.74499852902329, -9.168202490560926]",Entire home/apt,Lisbon,Portugal,38.744999,-9.168202,"Lisbon, Portugal",38.722252,-9.139337,2.211431
3,500,3025693,"[38.70144477298908, -9.240587547103432]",Private room,Lisbon,Portugal,38.701445,-9.240588,"Lisbon, Portugal",38.722252,-9.139337,5.644946
4,37,3063405,"[38.71109888002856, -9.148347820132365]",Entire home/apt,Lisbon,Portugal,38.711099,-9.148348,"Lisbon, Portugal",38.722252,-9.139337,0.910974


In [144]:
rooms_lat = []   # create empty lists
rooms_long = []
cities_lat = [] 
cities_long = []
calculated_distance = []

for index, row in cleanData.iterrows():
    room_lat1 = row['Room_Lat']
    room_long1 = row['Room_Long']
    
# Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(row, gkey)

#Run a request to endpoint and convert result to json
    cities_lat_long = requests.get(target_url).json()

    city_lat2 = cities_lat_long["results"][0]["geometry"]["location"]["lat"]
    city_long2 = cities_lat_long["results"][0]["geometry"]["location"]["lng"]
    
#CALCULATES DISTANCE OF ROOMS AND CITIES LAT AND LONGS
    def distance(room_long1, room_lat1, city_long2, city_lat2):
        room_long1, room_lat1, city_long2, city_lat2 = map(radians, [room_long1, room_lat1, city_long2, city_lat2])
        dlon = city_long2 - room_long1
        dlat = city_lat2 - room_lat1
        a = sin(dlat / 2) ** 2 + cos(room_lat1) * cos(city_lat2) * sin(dlon / 2) ** 2
        return 2 * 3958.756 * asin(sqrt(a)) #6371 km / 3958.756 for miles

    room_city_distance = distance(room_long1, room_lat1, city_long2, city_lat2)

    rooms_lat.append(room_lat1)
    rooms_long.append(room_long1)
    cities_lat.append(city_lat2)
    cities_long.append(city_long2)
    calculated_distance.append(room_city_distance)
    
print("rooms_lat:", rooms_lat)
print("rooms_long:", rooms_long)
print("cities_lat:", cities_lat)
print("cities_long:", cities_long)
print("calculated_distance:", calculated_distance)

rooms_lat: [38.74584282796673, 38.70213919994125, 38.74499852902329, 38.70144477298908, 38.71109888002856, 38.70824853381608, 38.751007598903335, 38.75894252695615, 38.746816666725785, 38.82631965777253, 38.71694216351822, 39.00386501446866, 38.72084577055854, 38.94581115726129, 38.711728978929635, 38.71308536425166, 38.71580738854452, 38.72775473777788, 38.70730938949486, 38.711248688937985, 38.97143166706441, 38.71715322999262, 38.72231541441824, 38.712737943557215, 38.726883479571555, 38.71062379871473, 38.795819430893125, 39.005962728530676, 38.7464481134842, 38.720637221426024, 38.75089940151426, 38.70530645655638, 38.690300201133766, 38.71248860445329, 38.71208442812792, 38.75765204552851, 38.71740795835646, 38.73086044505078, 38.71395646824652, 38.72385745438006, 38.72675852686154, 38.70914362687136, 38.710019718476204, 38.69055218940939, 38.71497145542667, 38.69463473046933, 38.69974647942219, 38.7084587440487, 38.72591566775218, 38.71470213658233, 38.87050144309989, 38.8146947

In [149]:
cleanData['City_Lat'] = cities_lat
cleanData['City_Long'] = cities_long
cleanData['Calc_Distance'] = calculated_distance

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Room_Lat,Room_Long,Selection,City_Lat,City_Long,Calc_Distance
0,125,2998105,"[38.74584282796673, -9.152359763007405]",Entire home/apt,Lisbon,Portugal,38.745843,-9.152360,"Lisbon, Portugal",38.722252,-9.139337,1.774650
1,500,3003712,"[38.70213919994125, -9.177363170810427]",Private room,Lisbon,Portugal,38.702139,-9.177363,"Lisbon, Portugal",38.722252,-9.139337,2.476753
2,650,3008031,"[38.74499852902329, -9.168202490560926]",Entire home/apt,Lisbon,Portugal,38.744999,-9.168202,"Lisbon, Portugal",38.722252,-9.139337,2.211431
3,500,3025693,"[38.70144477298908, -9.240587547103432]",Private room,Lisbon,Portugal,38.701445,-9.240588,"Lisbon, Portugal",38.722252,-9.139337,5.644946
4,37,3063405,"[38.71109888002856, -9.148347820132365]",Entire home/apt,Lisbon,Portugal,38.711099,-9.148348,"Lisbon, Portugal",38.722252,-9.139337,0.910974


In [152]:
cleanData= cleanData.drop(columns=["Coordinates"])
cleanData.head()

,Price,Room_Id,Room_Type,City,Country,Room_Lat,Room_Long,Selection,City_Lat,City_Long,Calc_Distance
0,125,2998105,Entire home/apt,Lisbon,Portugal,38.745843,-9.152360,"Lisbon, Portugal",38.722252,-9.139337,1.774650
1,500,3003712,Private room,Lisbon,Portugal,38.702139,-9.177363,"Lisbon, Portugal",38.722252,-9.139337,2.476753
2,650,3008031,Entire home/apt,Lisbon,Portugal,38.744999,-9.168202,"Lisbon, Portugal",38.722252,-9.139337,2.211431
3,500,3025693,Private room,Lisbon,Portugal,38.701445,-9.240588,"Lisbon, Portugal",38.722252,-9.139337,5.644946
4,37,3063405,Entire home/apt,Lisbon,Portugal,38.711099,-9.148348,"Lisbon, Portugal",38.722252,-9.139337,0.910974


In [123]:
#Export city names to csv
cleanData.to_csv('../data.csv', index=False)

# Import cities file as DataFrame
cleanData_df = pd.read_csv("../data.csv")
cleanData_df.head()

,Price,Room_Id,Room_Type,City,Country,Room_Lat,Room_Long,Selection,City_Lat,City_Long,Calc_Distance
0,125,2998105,Entire home/apt,Lisbon,Portugal,38.745843,-9.152360,"Lisbon, Portugal",38.722252,-9.139337,1.774650
1,500,3003712,Private room,Lisbon,Portugal,38.702139,-9.177363,"Lisbon, Portugal",38.722252,-9.139337,2.476753
2,650,3008031,Entire home/apt,Lisbon,Portugal,38.744999,-9.168202,"Lisbon, Portugal",38.722252,-9.139337,2.211431
3,500,3025693,Private room,Lisbon,Portugal,38.701445,-9.240588,"Lisbon, Portugal",38.722252,-9.139337,5.644946
4,37,3063405,Entire home/apt,Lisbon,Portugal,38.711099,-9.148348,"Lisbon, Portugal",38.722252,-9.139337,0.910974


In [53]:
# conn_str = "root:***password***@localhost/airbnb_data"
# engine = create_engine(f'mysql://{conn_str}')

# #check for tables in database
# #engine.table_names()

# #enter data into database
# ***df***.to_sql(name="airbnb_data_tbl", con=engine, if_exists="append",index=False)

In [ ]:
# this site confirmed the calculation is correct:
# https://www.gpsvisualizer.com/calculators